# Importar librerías

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Importar base de datos

In [6]:
df = pd.read_csv('Datasets/datos_limpios.csv')
df.job_level = df.job_level.astype(str)
df.stock_option_level = df.stock_option_level.astype(str)
df.training_times_last_year = df.training_times_last_year.astype(str)

In [109]:
df.shape

(4401, 27)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   employee_id                 4401 non-null   int64  
 1   age                         4401 non-null   int64  
 2   attrition                   4401 non-null   object 
 3   business_travel             4401 non-null   object 
 4   department                  4401 non-null   object 
 5   distance_from_home          4401 non-null   int64  
 6   education                   4401 non-null   object 
 7   education_field             4401 non-null   object 
 8   gender                      4401 non-null   object 
 9   job_level                   4401 non-null   object 
 10  job_role                    4401 non-null   object 
 11  marital_status              4401 non-null   object 
 12  monthly_income              4401 non-null   int64  
 13  num_companies_worked        4401 

# Análisis de correlación

## Correlación de variables numéricas

Seleccionar variables numéricas

In [111]:
df_numericas = df.select_dtypes(include=['int64', 'float64'])

Obtener matriz de correlaciones

In [112]:
corr_matrix = df_numericas.drop(['employee_id'], axis=1).corr()
corr_matrix = corr_matrix.apply(lambda x: np.round(x, 2))
corr_matrix

,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,years_at_company,years_since_last_promotion,years_with_curr_manager,mean_time
age,1.00,0.01,-0.04,0.30,-0.03,0.68,0.31,0.22,0.20,-0.00
distance_from_home,0.01,1.00,-0.02,-0.01,0.04,0.01,0.03,0.00,0.02,0.01
monthly_income,-0.04,-0.02,1.00,-0.02,0.00,-0.03,0.00,0.06,0.02,-0.01
num_companies_worked,0.30,-0.01,-0.02,1.00,0.03,0.24,-0.12,-0.04,-0.11,-0.02
percent_salary_hike,-0.03,0.04,0.00,0.03,1.00,-0.02,-0.03,-0.03,-0.04,0.03
total_working_years,0.68,0.01,-0.03,0.24,-0.02,1.00,0.63,0.40,0.46,-0.01
years_at_company,0.31,0.03,0.00,-0.12,-0.03,0.63,1.00,0.62,0.77,-0.02
years_since_last_promotion,0.22,0.00,0.06,-0.04,-0.03,0.40,0.62,1.00,0.51,-0.01
years_with_curr_manager,0.20,0.02,0.02,-0.11,-0.04,0.46,0.77,0.51,1.00,-0.03
mean_time,-0.00,0.01,-0.01,-0.02,0.03,-0.01,-0.02,-0.01,-0.03,1.00


Mapa de calor de correlaciones

In [120]:
fig = px.imshow(corr_matrix, text_auto=True)
fig.layout.update(height=500)
fig.show()

Las correlaciones más significativas son:

- Entre edad y total de años trabajando. Con una correlación de 0.68, tiene sentido que los empleados de más edad sean los que lleven más años trabajando.

- Entre total de años trabajando y total de años en la compañía. Los empleados que llevan más años como empleados tienden a ser los que llevan más años en la compañía.

- Entre años en la compañía y años desde la última promoción. Entre más tiempo tenga el empleado en la compañía es más probable que haya pasado mucho tiempo desde su último ascenso.

- Entre años en la compañía y años al mando del jefe actual.

Todas las variables relacionadas con el tiempo de estancia del empleado en la empresa tienen una correlación alta. 'years_at_company' y 'years_with_curr_manager' se pueden considerar variables redundantes y sería bueno eliminar una de estas, veremos qué resultados obtenemos con otros métodos de selección de variables como la regularización.

## Correlación entre la variable objetivo y variables categóricas

In [47]:
def corr_cat(df, var1, var2):
    from scipy.stats import chi2_contingency
    tabla_contingencia = pd.crosstab(df[var1], df[var2])
    display(tabla_contingencia)
    chi2, p, _, _ = chi2_contingency(tabla_contingencia)
    print('Estadístico de Chi-cuadrado:', chi2)
    print('Valor p:', p)
    alpha = 0.05  # Nivel de significancia
    if p < alpha:
        print(f"Hay una asociación significativa entre {var1} y {var2}.")
    else:
        print(f"No hay evidencia de una asociación significativa entre {var1} y {var2}.")

In [48]:
corr_cat(df,'attrition','business_travel')


business_travel,Non_Travel,Travel_Frequently,Travel_Rarely
attrition,,,
No,413,623,2656
Yes,36,206,467


Estadístico de Chi-cuadrado: 71.69435279452458
Valor p: 2.702521133933196e-16
Hay una asociación significativa entre attrition y business_travel.


In [49]:
corr_cat(df,'attrition','department')

department,Human Resources,Research & Development,Sales
attrition,,,
No,132,2425,1135
Yes,56,452,201


Estadístico de Chi-cuadrado: 27.483175954829832
Valor p: 1.0767234429318749e-06
Hay una asociación significativa entre attrition y department.


In [51]:
corr_cat(df,'attrition','education')

education,Doctor,Licenciado,Máster,Por debajo de la universidad,Universitario
attrition,,,,,
No,123,1446,1005,432,686
Yes,21,265,186,78,159


Estadístico de Chi-cuadrado: 5.783649417345585
Valor p: 0.21589856582779615
No hay evidencia de una asociación significativa entre attrition y education.


In [52]:
corr_cat(df,'attrition','education_field')

education_field,Human Resources,Life Sciences,Marketing,Medical,Other,Technical Degree
attrition,,,,,,
No,48,1510,402,1165,216,351
Yes,33,302,75,224,30,45


Estadístico de Chi-cuadrado: 46.22025199807734
Valor p: 8.19104430706428e-09
Hay una asociación significativa entre attrition y education_field.


In [16]:
corr_cat(df,'attrition','gender')

gender,Female,Male
attrition,,
No,1492,2200
Yes,269,440


Estadístico de Chi-cuadrado: 1.4117417191086203
Valor p: 0.23476747305053552
No hay evidencia de una asociación significativa entre attrition y gender.


In [54]:
corr_cat(df,'attrition','job_level')

job_level,1,2,3,4,5
attrition,,,,,
No,1373,1315,557,267,180
Yes,251,284,96,51,27


Estadístico de Chi-cuadrado: 6.14021948275271
Valor p: 0.1889184612815812
No hay evidencia de una asociación significativa entre attrition y job_level.


In [55]:
corr_cat(df,'attrition','job_role')

job_role,Healthcare Representative,Human Resources,Laboratory Technician,Manager,Manufacturing Director,Research Director,Research Scientist,Sales Executive,Sales Representative
attrition,,,,,,,,,
No,336,135,649,264,384,183,717,812,212
Yes,57,21,125,42,48,56,159,165,36


Estadístico de Chi-cuadrado: 23.917757147865213
Valor p: 0.0023656902080819967
Hay una asociación significativa entre attrition y job_role.


In [56]:
corr_cat(df,'attrition','marital_status')

marital_status,Divorced,Married,Single
attrition,,,
No,879,1763,1050
Yes,99,251,359


Estadístico de Chi-cuadrado: 137.28138802271818
Valor p: 1.5478374736327779e-30
Hay una asociación significativa entre attrition y marital_status.


In [20]:
corr_cat(df,'attrition','stock_option_level')

stock_option_level,0,1,2,3
attrition,,,,
No,1571,1516,389,216
Yes,318,268,84,39


Estadístico de Chi-cuadrado: 3.371942516539834
Valor p: 0.33775444661956466
No hay evidencia de una asociación significativa entre attrition y stock_option_level.


In [21]:
corr_cat(df,'attrition','environment_satisfaction')

environment_satisfaction,Alto,Bajo,Medio,Muy alto
attrition,,,,
No,1182,631,726,1153
Yes,190,213,127,179


Estadístico de Chi-cuadrado: 65.19092150371308
Valor p: 4.5658311002337154e-14
Hay una asociación significativa entre attrition y environment_satisfaction.


In [22]:
corr_cat(df,'attrition','job_satisfaction')

job_satisfaction,Alto,Bajo,Medio,Muy alto
attrition,,,,
No,1100,663,701,1228
Yes,218,197,137,157


Estadístico de Chi-cuadrado: 52.97336408099865
Valor p: 1.857491694476218e-11
Hay una asociación significativa entre attrition y job_satisfaction.


In [23]:
corr_cat(df,'attrition','work_life_balance')

work_life_balance,Buena,La mejor,Mala,Muy buena
attrition,,,,
No,848,373,164,2307
Yes,171,81,75,382


Estadístico de Chi-cuadrado: 49.799083547955505
Valor p: 8.816361549528376e-11
Hay una asociación significativa entre attrition y work_life_balance.


In [24]:
corr_cat(df,'attrition','job_involvement')

job_involvement,Alto,Bajo,Medio,Muy alto
attrition,,,,
No,2202,193,944,353
Yes,397,54,180,78


Estadístico de Chi-cuadrado: 8.655394731878292
Valor p: 0.03424144376108539
Hay una asociación significativa entre attrition y job_involvement.


In [25]:
corr_cat(df,'attrition','performance_rating')

performance_rating,Excelente,Sobresaliente
attrition,,
No,3139,553
Yes,586,123


Estadístico de Chi-cuadrado: 2.390737339890053
Valor p: 0.12205604712347845
No hay evidencia de una asociación significativa entre attrition y performance_rating.
